# Day 20: Race Condition

https://adventofcode.com/2024/day/20

## --- Part One ---

In [179]:
import numpy as np
import math
import re
import heapq
from copy import deepcopy
from pprint import pprint
from termcolor import colored

file = 'input.txt'
file = 'sample.txt'
# file = 'sample2.txt'

# init vars
answer = 0
maze = []

# open file & load content
with open(file, 'r') as f:
    maze = np.array([list(line) for line in f.read().splitlines()])

# we re-use a simplified dijkstra search alg from day 16
def dijkstra(grid, start, goal):
    rows, cols = grid.shape
    distances = np.full((rows, cols), np.inf)
    distances[start] = 0
    priority_queue = [(0, start)]
    visited = set()

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        if current_node in visited:
            continue
        visited.add(current_node)
        
        if current_node == goal:
            break
        
        neighbors = get_neighbors(current_node, rows, cols)
        for neighbor in neighbors:
            if grid[neighbor] == 1:  # Skip obstacles
                continue
            distance = current_distance + 1  # Assuming uniform cost
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))
    
    return distances

def get_neighbors(node, rows, cols):
    x, y = node
    neighbors = []
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols:
            neighbors.append((nx, ny))
    return neighbors

start = (np.where(maze == 'S')[0][0],np.where(maze == 'S')[1][0])
end = (np.where(maze == 'E')[0][0],np.where(maze == 'E')[1][0])
maze_weights = np.ones_like(maze, dtype=int)

# set the walls
maze_weights[np.where(maze !='#')] = 0

path = dijkstra(maze_weights, start, end)
path_length = (np.max(path[np.isfinite(path)]))
    
# print(path)

In [180]:
# path_colums = [path[:, i] for i in range(path.shape[1])]
# Function to get the absolute difference between two integers in a list
def absolute_difference(num1, num2):
    return abs(num1- num2)

answer = 0

# check for cheats in rows
for row_idx,row in enumerate(path):
    # skip first and last row
    if row_idx == 0 or row_idx == len(path)-1:
        continue
    for i in range(1,len(row)-2):
        cur = row[i]
        # only check the numbers not the walls
        if np.isfinite(cur):
            if row[i+1] == np.inf and np.isfinite(row[i+2]):
                shortcut = [int(row[i]), int(row[i+2])]
                time_saved = absolute_difference(shortcut[0],shortcut[1]) -2
                if time_saved >= 100:
                    answer +=1

path_cols = [path[:, i] for i in range(path.shape[1])]

for row_idx,row in enumerate(path_cols):
    # skip first and last row
    if row_idx == 0 or row_idx == len(path)-1:
        continue
    for i in range(1,len(row)-2):
        cur = row[i]
        # only check the numbers not the walls
        if np.isfinite(cur):
            if row[i+1] == np.inf and np.isfinite(row[i+2]):
                shortcut = [int(row[i]), int(row[i+2])]
                time_saved = absolute_difference(shortcut[0],shortcut[1]) -2
                if time_saved >= 100:
                    answer +=1

print('Answer to part 1:', answer)
            

Answer to part 1: 0


## --- Part Two ---

In [289]:
import numpy as np
import math
import sys
import re
import heapq
from copy import deepcopy
from pprint import pprint
from termcolor import colored

file = 'input.txt'
# file = 'sample.txt'
# file = 'sample2.txt'

# init vars
answer = 0
maze = []

# open file & load content
with open(file, 'r') as f:
    maze = np.array([list(line) for line in f.read().splitlines()])

# we re-use a simplified dijkstra search alg from day 16
def dijkstra(grid, start, goal):
    rows, cols = grid.shape
    distances = np.full((rows, cols), np.inf)
    distances[start] = 0
    priority_queue = [(0, start)]
    visited = set()

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        if current_node in visited:
            continue
        visited.add(current_node)
        
        if current_node == goal:
            break
        
        neighbors = get_neighbors(current_node, rows, cols)
        for neighbor in neighbors:
            if grid[neighbor] == 1:  # Skip obstacles
                continue
            distance = current_distance + 1  # Assuming uniform cost
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))
    
    return distances

def get_neighbors(node, rows, cols):
    x, y = node
    neighbors = []
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols:
            neighbors.append((nx, ny))
    return neighbors

start = (np.where(maze == 'S')[0][0],np.where(maze == 'S')[1][0])
end = (np.where(maze == 'E')[0][0],np.where(maze == 'E')[1][0])
maze_weights = np.ones_like(maze, dtype=int)

# set the walls
maze_weights[np.where(maze !='#')] = 0

path = dijkstra(maze_weights, start, end)
path_length = (np.max(path[np.isfinite(path)]))

In [297]:

def print_maze(array):
    for a in array:
        # print(''.join(a.astype(str).tolist()))
        print(a.tolist())

def get_surrounding_area(arr, center, size):
    """
    Get the surrounding area from a larger 2D numpy array in a particular location.
    
    Parameters:
    arr (np.ndarray): The larger 2D numpy array.
    center (tuple): The (row, column) index of the center location.
    size (int): The size of the surrounding area to extract (must be odd).
    
    Returns:
    np.ndarray: The surrounding area as a 2D numpy array.
    """
    row, col = center
    half_size = size // 2
    
    # Calculate the start and end indices for rows and columns
    row_start = max(row - half_size, 0)
    row_end = min(row + half_size + 1, arr.shape[0])
    col_start = max(col - half_size, 0)
    col_end = min(col + half_size + 1, arr.shape[1])
    
    return arr[row_start:row_end, col_start:col_end]

# minimum time save
inverted_maze = np.where(maze_weights == 0, 1, 0)

if file != 'input.txt':
    max_cheat_length = np.inf

answer2 = 0

# dont truncate numpy output
np.set_printoptions(threshold=sys.maxsize)

# we re-use a simplified dijkstra search alg from day 16
def dijkstra2(grid, start, goal):
    grid = deepcopy(grid)
    rows, cols = grid.shape
    distances = np.full((rows, cols), np.inf)
    distances[start] = 0
    grid[start] = 0
    grid[goal] = 0
    priority_queue = [(0, start)]
    visited = set()

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        if current_node in visited:
            continue
        visited.add(current_node)
        
        if current_node == goal:
            break

        # if current_distance > max_cheat_length:
        #     break
        
        neighbors = get_neighbors(current_node, rows, cols)
        for neighbor in neighbors:
            if grid[neighbor] == 1:  # Skip obstacles
                continue
            distance = current_distance + 1  # Assuming uniform cost
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))
    
    return distances[goal]

inverted_maze = np.zeros_like(maze,dtype=int)

# EXCLUDE BORDER CELLS
inverted_maze[0,:] = 1
inverted_maze[len(inverted_maze)-1,:] = 1
inverted_maze[:,0] = 1
inverted_maze[:,len(inverted_maze[0])-1] = 1


# print(inverted_maze)
print(path_length)


for i in range(0,int(path_length)):
    
    # ---- checking and displaying for loop loading times ----
    # Calculate the percentage
    percent_complete = (i + 1) / path_length * 100
    # Print the progress
    # print(f"Loading... {percent_complete:.2f}% and found {answer2} so far", end='\r')
    # --------------------------------------------------------

    loc = (np.where(path == i)[0][0],np.where(path == i)[1][0])
    # potential_cuts = np.where((path != np.inf) & (path>= (i +100)))
    potential_cuts = np.where((path != np.inf) & (path>= (i+100)))
    print(potential_cuts[0][1], potential_cuts[1][1])
    raise
    # ===== LOCAL CUTS =========
    # local_maze = get_surrounding_area(path, loc, max_cheat_length*2)
    # new_loc = (np.where(local_maze == i)[0][0],np.where(local_maze == i)[1][0])
    # local_inv_maze = get_surrounding_area(inverted_maze, loc, max_cheat_length*2)
    # potential_cuts = np.where((local_maze != np.inf) & (local_maze>= i +100))
    # ==========
    for i in range(0,len(potential_cuts[0])):
        end = (potential_cuts[0][i], potential_cuts[1][i])
        local_cheat_path_length = dijkstra2(inverted_maze, loc, end)
        diff = absolute_difference(path[loc],path[end])
        time_saved = diff-local_cheat_path_length
        if local_cheat_path_length != np.inf and time_saved >100:
            # print('start',[int(loc[0]),int(loc[1])],'end', [int(end[0]),int(end[1])], int(path[loc]),'-->',int(path[end]), int(local_cheat_path_length),time_saved )
            answer2 +=1

print('\nAnswer to part 2:', answer2)

9316.0
1 2


RuntimeError: No active exception to reraise